In [4]:
# Load libraries

import os
import pandas as pd
import datetime as dt
import numpy as np
import re
from pathlib import Path
import glob

In [5]:
# Insert ZIP CODEs & Cities to filter
zip_codes = [80501, 80020, 80602, 80919, 80642]
cities = []
requested_home_value = () 
if requested_home_value == ():
    requested_home_value = 200000
    

In [6]:
# Load CSV file to filter
excel_files = list(Path.cwd().glob('*.xlsx'))
if not excel_files:
	raise FileNotFoundError(f"No Excel files found in {Path.cwd()}")

# Merge all Excel files into a single DataFrame
data = pd.concat((pd.read_excel(f) for f in excel_files), ignore_index=True)

# Get the CSV file name without ".csv" extension 
excel_file_name = excel_files[0].stem

# Get the folder name
folder_name = Path.cwd().name

# Remove unnecesary columns
data = data[[
	'FirstName', 'LastName', 'Gender', 'Address', 'City', 'State', 'ZipCode',	
	'PhoneNumber', 'MobileNumber', 'HomeownerConfirmed', 'HomeValue', 
	'LengthOfResidence', 'EmailAddress', 'Latitude', 'Longitude'
]]

# Store the original number of rows
original_row_count = len(data)

# Print the original number of rows
print(f"Original number of rows in this merged list is: {original_row_count}")

Original number of rows in this merged list is: 1481139


In [7]:
# Show a sample of the data
data.sample(20)

,FirstName,LastName,Gender,Address,City,State,ZipCode,PhoneNumber,MobileNumber,HomeownerConfirmed,HomeValue,LengthOfResidence,EmailAddress,Latitude,Longitude
439061,Maynor,Sanchez,Male,1765 Lansing St,Aurora,CO,80010,NaN,NaN,Y,"$23,000",3.0,NaN,39.745020,-104.857900
1013792,Chu,Kim,Female,9936 Raleigh St,Westminster,CO,80031,NaN,NaN,Y,"$58,000",1.0,kim11965@yahoo.com,39.877550,-105.040720
1048430,Keith,Schulteis,Male,4301 S Pierce St Apt 5D,Littleton,CO,80123,NaN,NaN,Y,"$477,000",4.0,NaN,39.636690,-105.072490
661817,Darren,Cutter,Male,7130 S Eudora Ct,Centennial,CO,80122,NaN,NaN,Y,"$46,000",NaN,NaN,39.586980,-104.930510
123856,Alex,Cordova,Unknown,220 Wark Ave,Berthoud,CO,80513,NaN,NaN,Y,"$38,000",1.0,alexpcordova@yahoo.com,40.301633,-105.217158
564181,Andrew,Hagen,Male,1575 Ivanhoe St,Denver,CO,80220,NaN,2.259218e+09,Y,"$50,000",NaN,NaN,39.741560,-104.921439
441781,April,Mann,Female,11063 Claude Ct,Northglenn,CO,80233,NaN,NaN,Y,"$26,000",15.0,NaN,39.897099,-104.959600
317256,Lars,Strunk,Male,11630 E Montana Pl,Aurora,CO,80012,NaN,7.202993e+09,Y,"$34,000",15.0,NaN,39.683659,-104.853150
387209,Maricela,Mares,Female,6000 Rose Ln,Commerce City,CO,80022,NaN,NaN,Y,"$22,000",10.0,NaN,39.805769,-104.918370
555723,Joo,Lee,Unknown,13365 Race St,Thornton,CO,80241,NaN,NaN,Y,"$39,000",15.0,NaN,39.936720,-104.964549


In [8]:
# Verify the empty cells in each column
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481139 entries, 0 to 1481138
Data columns (total 15 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   FirstName           1481138 non-null  object 
 1   LastName            1481139 non-null  object 
 2   Gender              1481139 non-null  object 
 3   Address             1481139 non-null  object 
 4   City                1481139 non-null  object 
 5   State               1481139 non-null  object 
 6   ZipCode             1481139 non-null  int64  
 7   PhoneNumber         122893 non-null   float64
 8   MobileNumber        689049 non-null   float64
 9   HomeownerConfirmed  1481139 non-null  object 
 10  HomeValue           1336537 non-null  object 
 11  LengthOfResidence   1273020 non-null  float64
 12  EmailAddress        595425 non-null   object 
 13  Latitude            1481139 non-null  float64
 14  Longitude           1481139 non-null  float64
dtypes: float64(5), 

In [9]:
            ### Filtering Process starts here ###

# Filter out rows without phone numbers and cell numbers
data = data[data['PhoneNumber'].notna() | data['MobileNumber'].notna()]

# Store the number of rows after removing rows without phone/cell numbers
filter_1_row_count = len(data)

# Print the number of rows after removing rows without phone/cell numbers
print(f"Number of rows after removing rows without phone and cell numbers: {filter_1_row_count}")
print()
print(f"Number of rows removed in this step: {original_row_count - filter_1_row_count}")

Number of rows after removing rows without phone and cell numbers: 750398

Number of rows removed in this step: 730741


In [10]:
# Define the filters for addresses
filters = [
' Apt ',
' Ste ',
' Lot ',
' Spc ',
'Unit ', 
'Trlr ',
'-',
'#',
'Mobi ',
'Po Box',
' Ph ',
' Flr ',
]

# Remove rows where 'Address' contains any of the filter strings 
for filter_str in filters:
    data = data[~data['Address'].str.contains(filter_str, case=False, na=False)]

# Store the number of rows after filtering
filter_2_row_count = len(data)

# Print the number of rows after filtering
print(f"Number of rows after filtering addresses: {filter_2_row_count}")
print()
print(f"Number of rows removed in this step: {filter_1_row_count - filter_2_row_count}")

Number of rows after filtering addresses: 664078

Number of rows removed in this step: 86320


In [11]:
# Filter out rows with empty 'Address'
data = data[data['Address'].notna()]

# Store the number of rows after removing empty addresses
filter_3_row_count = len(data)

# Print the number of rows after removing empty addresses
print(f"Number of rows after removing empty addresses: {filter_3_row_count}")
print()
print(f"Number of rows removed in this step: {filter_2_row_count - filter_3_row_count}")

Number of rows after removing empty addresses: 664078

Number of rows removed in this step: 0


In [12]:
# Filter the data based on ZIP CODEs and Cities
# Apply zip code filter if zip_codes is a non-empty list
if zip_codes not in (None, [], '') and len(zip_codes) > 0:
    # Ensure zip codes are strings for comparison
    zip_codes_str = [str(z) for z in zip_codes]
    data = data[data['ZipCode'].astype(str).isin(zip_codes_str)]

# Apply city filter if cities is a non-empty list
if cities not in (None, [], '') and len(cities) > 0:
    # Remove empty/None entries from cities list
    cities_clean = [c for c in cities if c not in (None, '')]
    if cities_clean:
        data = data[data['City'].isin(cities_clean)]

# Store the number of rows after filtering ZIP CODEs and Cities
filter_4_row_count = len(data)

# Print the number of rows after filtering ZIP CODEs and Cities
print(f"Number of rows after filtering ZIP CODEs and Cities: {filter_4_row_count}")
print()
print(f"Number of rows removed in this step: {filter_3_row_count - filter_4_row_count}")

Number of rows after filtering ZIP CODEs and Cities: 46645

Number of rows removed in this step: 617433


In [13]:
# Create a new dataframe to hold rows without home values
missing_homevalue = data[data['HomeValue'].isna()]

# Make HomeValue column numeric
data['HomeValue'] = data['HomeValue'].astype(str).str.replace('[\\$,]', '', regex=True)

# Filter out rows where HomeValue is less than the specified amount
data['HomeValue'] = pd.to_numeric(data['HomeValue'], errors='coerce') # Convert to numeric, setting errors to NaN
data = data[data['HomeValue'] >= requested_home_value] # Keep only rows with HomeValue >= than requested_home_value
data['HomeValue'] = data['HomeValue'].astype('int64') # Convert HomeValue back to int64

# Store the number of rows after filtering HomeValue
filter_5_row_count = len(data)

# Print the number of rows after filtering HomeValue
print(f"Number of rows after filtering HomeValue >= {requested_home_value}: {filter_5_row_count}")
print()
print(f"Number of rows removed in this step: {filter_4_row_count - filter_5_row_count}")

Number of rows after filtering HomeValue >= 200000: 8364

Number of rows removed in this step: 38281


In [14]:
print(f"Number of rows with missing HomeValue stored separately: {len(missing_homevalue)}")

Number of rows with missing HomeValue stored separately: 1538


In [15]:
# Convert 'PhoneNumber' and 'MobileNumber' to integer format
# Use nullable Int64 to handle NaN values
data['PhoneNumber'] = (pd.to_numeric(data['PhoneNumber'], errors='coerce').astype('Int64'))
data['MobileNumber'] = (pd.to_numeric(data['MobileNumber'], errors='coerce').astype('Int64'))

# Verify the data types
data.sample(15)

,FirstName,LastName,Gender,Address,City,State,ZipCode,PhoneNumber,MobileNumber,HomeownerConfirmed,HomeValue,LengthOfResidence,EmailAddress,Latitude,Longitude
1450752,Vicki,Maketa,Female,5480 Cordillera Ct,Colorado Spgs,CO,80919,<NA>,7193399260,Y,1161000,10.0,BMAKETA@HOTMAIL.COM,38.909800,-104.835060
1465024,Virginia,Gibney,Female,1180 War Eagle Dr N,Colorado Spgs,CO,80919,<NA>,7192002396,Y,781000,NaN,NaN,38.925650,-104.848810
1434019,Regina,Kornmesser,Female,970 Golden Hills Rd,Colorado Spgs,CO,80919,<NA>,7197617405,Y,709000,18.0,NaN,38.908240,-104.844479
1464279,Mara,Abernethy,Female,6525 Ashton Park Pl,Colorado Spgs,CO,80919,<NA>,7193231405,Y,777000,15.0,NaN,38.924749,-104.867250
6723,Larry,Rumery,Male,20497 County Road 4,Hudson,CO,80642,<NA>,3039095891,Y,633000,15.0,NaN,40.017860,-104.671050
1479232,Linda,Tanner,Female,745 Saddlemountain Rd,Colorado Spgs,CO,80919,<NA>,7025407507,Y,389000,5.0,LINDATANNER7@AOL.COM,38.923281,-104.840198
1452227,Natalia,Valentine,Female,7950 Ruststone Ct,Colorado Spgs,CO,80919,7195592880,<NA>,Y,988000,1.0,RITA.VALENTINE@YAHOO.COM,38.945040,-104.884009
1438392,Sharon,McAllister,Female,672 Blackhawk Dr,Colorado Spgs,CO,80919,7195909361,<NA>,Y,691000,15.0,GABRIEL_KELLY13@YAHOO.COM,38.931940,-104.840930
1464878,Matthew,Evans,Male,1585 Territory Trl,Colorado Spgs,CO,80919,<NA>,7402155395,Y,277000,4.0,exposure30k@gmail.com,38.905850,-104.853159
1453735,John,Blythe,Male,5380 Sevenoaks Dr,Colorado Spgs,CO,80919,<NA>,3032299265,Y,659000,15.0,NaN,38.909970,-104.857580


In [16]:
# Sepparate mobile numbers into a different df and name phone numbers as Alt.Phone

cell_numbers = data[data['MobileNumber'].notna()].copy() # Get rows with MobileNumber not null
cell_numbers.rename(columns={'PhoneNumber': 'Alt.Phone', 'MobileNumber': 'PhoneNumber'}, inplace=True) # Rename columns

# Store the lenght of the dataframe before removing duplicates
duplicates_filter = len(cell_numbers)

# Remove duplicates based on 'PhoneNumber'column
cell_numbers = cell_numbers.drop_duplicates(subset=['PhoneNumber'], keep='first').reset_index(drop=True)

# Return the home value column to its original format
cell_numbers['HomeValue'] = cell_numbers['HomeValue'].apply(lambda x: f"${x:,.0f}")

# Show the length of the cell phones list
print(f"Number of rows with cell phone numbers after removing duplicates: {len(cell_numbers)}")
print()
print(f"Number of rows removed in this step: {duplicates_filter - len(cell_numbers)}")

Number of rows with cell phone numbers after removing duplicates: 7158

Number of rows removed in this step: 241


In [17]:
# Create the land lines list
land_lines = data[data['MobileNumber'].isna()].copy() # Get rows with MobileNumber null

# Store the lenght of the dataframe before removing duplicates
duplicates_filter_land = len(land_lines)

# Remove duplicates based on 'PhoneNumber'column
land_lines = land_lines.drop_duplicates(subset=['PhoneNumber'], keep='first').reset_index(drop=True)

# Show the length of the land lines list
print(f"Number of rows with land line numbers after removing duplicates: {len(land_lines)}")
print()
print(f"Number of rows removed in this step: {duplicates_filter_land - len(land_lines)}")

Number of rows with land line numbers after removing duplicates: 965

Number of rows removed in this step: 0


In [ ]:
# Name the output files
output_file_landlines = f"{excel_file_name}-Land.csv"
output_file_cellnumbers = f"{excel_file_name}-Cell.csv"

# Create a new directory for output files if it doesn't exist
output_dir = Path.cwd() / f"{folder_name} - FILTER OUTPUT"

# Ensure output directory exists
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Output directory created successfully as: {output_dir}")

Output directory created successfully as: c:\Users\ADMIN\Documents\RECORDS LISTS\MTN CO\MTN CO- FILTER OUTPUT


In [19]:
# Save the filtered data to new CSV files

land_lines.to_csv(output_dir / output_file_landlines, index=False)  # Save land lines data
cell_numbers.to_csv(output_dir / output_file_cellnumbers, index=False) # Save cell numbers data
missing_homevalue.to_csv(output_dir / f"{excel_file_name}_Missing_HomeValue.csv", index=False) # Save missing home value data

print(f"Saved {len(land_lines)} land lines to {output_dir / output_file_landlines}")
print(f"Saved {len(cell_numbers)} cell numbers to {output_dir / output_file_cellnumbers}")
print(f"Saved {len(missing_homevalue)} rows with missing HomeValue to {output_dir / f'{excel_file_name}_Missing_HomeValue.csv'}")

Saved 965 land lines to c:\Users\ADMIN\Documents\RECORDS LISTS\MTN CO\MTN CO- FILTER OUTPUT\MTN-CO-ZIPS80501-80020-80602-80919-80642-12-11-25-1_LL_Filtered.csv
Saved 7158 cell numbers to c:\Users\ADMIN\Documents\RECORDS LISTS\MTN CO\MTN CO- FILTER OUTPUT\MTN-CO-ZIPS80501-80020-80602-80919-80642-12-11-25-1_Cell_Filtered.csv
Saved 1538 rows with missing HomeValue to c:\Users\ADMIN\Documents\RECORDS LISTS\MTN CO\MTN CO- FILTER OUTPUT\MTN-CO-ZIPS80501-80020-80602-80919-80642-12-11-25-1_Missing_HomeValue.csv
